In [1]:
from math import sqrt
from operator import itemgetter

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn import metrics
import sys
sys.path.append("..")
from AF.AF import AverageFilling

In [14]:
class UserBasedCollaborativeFilter():
    def __init__(self, records_train, records_test):
        records = np.vstack([records_train, records_test])
        n = len(np.unique(np.sort(records[:, 0])))
        m = len(np.unique(np.sort(records[:, 1])))

        # Initialize R
        R = np.zeros([n, m])
        for record in records_train:
            R[record[0], record[1]] = record[2]
        
        # Initialize indicator
        y = np.where(R, 1, 0)
        y_user = np.sum(y, axis=1)
        y_item = np.sum(y, axis=0)
        
        # Initialize 
        

In [13]:
records_train = np.loadtxt('../data/ml-100k/u1.base', dtype=np.int32)
records_test = np.loadtxt('../data/ml-100k/u1.test', dtype=np.int32)
records_train[:, :2] -= 1
records_test[:, :2] -= 1
rating_test = records_test[:, 2]



In [16]:
# usercf = UserBasedCollaborativeFilter(records_train, records_test)



5 5
